# 多步骤金融推理系统

本 Notebook 演示如何用 **LangChain** 构建一个多步骤、动态决策的金融推理系统，
结合 **MultiRouteChain** + **条件路由** + **模糊信息补全** + **市场动态查询** + **冲突检测**。

### 系统架构
```
[用户输入]
   ↓
[意图解析 & RouterChain]
   ├── 股票策略链（Stock Strategy Chain）
   ├── 债券策略链（Bond Strategy Chain）
   └── 澄清链（Clarification Chain）
   ↓
[市场信息动态获取]
   ↓
[冲突检测与最终建议]
```


In [ ]:
!pip install langchain langchain-openai

In [ ]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains.router import MultiRouteChain, RouterChain
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

# 初始化 LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# ===== 子链定义 =====
# 股票策略链
stock_prompt = PromptTemplate.from_template(
    "你是一名股票投资顾问。用户需求：{input}\n"
    "请结合当前市场信息（假设科技股上涨）生成股票投资建议。"
)
stock_chain = LLMChain(llm=llm, prompt=stock_prompt)

# 债券策略链
bond_prompt = PromptTemplate.from_template(
    "你是一名债券投资顾问。用户需求：{input}\n"
    "请结合当前市场信息（假设利率下降）生成债券投资建议。"
)
bond_chain = LLMChain(llm=llm, prompt=bond_prompt)

# 澄清链（模糊信息处理）
clarify_prompt = PromptTemplate.from_template(
    "用户输入不够明确：{input}\n"
    "请提出三个关键问题，帮助你更好地理解用户的投资需求。"
)
clarify_chain = LLMChain(llm=llm, prompt=clarify_prompt)

# ===== RouterChain 定义 =====
destinations = {
    "股票": stock_chain,
    "债券": bond_chain,
    "澄清": clarify_chain
}

router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(
    destinations="\n".join([f"{name}: {desc}" for name, desc in {
        "股票": "用户想投资股票或股市相关产品",
        "债券": "用户想投资债券或固定收益产品",
        "澄清": "用户输入模糊，需要先澄清需求"
    }.items()])
)
router_prompt = PromptTemplate(template=router_template, input_variables=["input"])
router_chain = LLMChain(llm=llm, prompt=router_prompt)

# ===== MultiRouteChain 组装 =====
multi_route_chain = MultiRouteChain(
    router_chain=RouterChain.from_llm(llm, router_prompt),
    destination_chains=destinations,
    default_chain=clarify_chain,
    verbose=True
)

# ===== 冲突检测函数 =====
def conflict_detection(advice: str) -> str:
    # 简单模拟冲突检测：如果同时提到“高风险”和“保守”，判定为冲突
    if "高风险" in advice and "保守" in advice:
        return advice + "\n⚠️ 检测到风险偏好冲突，请重新评估投资策略。"
    return advice

# ===== 测试运行 =====
user_inputs = [
    "我想找一些增长潜力高的科技公司股票",
    "我更喜欢安全稳定的债券产品",
    "我想投资但还不确定方向"
]

for text in user_inputs:
    print(f"\n=== 用户输入: {text} ===")
    advice = multi_route_chain.run({"input": text})
    checked_advice = conflict_detection(advice)
    print(checked_advice)
